# 🎓 Введение

Привет, участники соревнования! 👋

Этот ноутбук — ваш проводник в мир анализа временных рядов и построения моделей машинного обучения. Он написан специально для тех, кто только начинает знакомство с Kaggle и участвует в соревнованиях впервые.

Здесь мы вместе:
- 📈 научимся извлекать признаки из временных рядов,
- 🧠 построим базовую модель машинного обучения (LightGBM),
- 🧪 проведём корректную валидацию,
- 📤 подготовим прогноз и отправим его в виде submission-файла.

> 💡 Несмотря на то, что текущее решение не побеждает бенчмарки, **совсем небольшие улучшения** (например, другие признаки, параметры модели или архитектура) уже позволят вам занять **топовые места** в рейтинге!

Цель этого ноутбука — **показать вам базовый, но правильный путь**:
- никаких "магических" трюков,
- только понятные и воспроизводимые шаги.

Удачи в соревновании! 🚀  
**Вы уже на правильном пути.**


In [1]:
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import os

# 1. Загрузка
# work_dir = "../input/"
work_dir = "../input/kazakhstan-ai-respa-take-home/"
train = pd.read_csv(os.path.join(work_dir, "train.csv"))
train['submitted_date'] = pd.to_datetime(train['submitted_date'])
train.head()

,category,submitted_date,num_papers
0,hep-ph - High Energy Physics - Phenomenology,2000-01-01,5
1,hep-ph - High Energy Physics - Phenomenology,2000-01-02,6
2,hep-ph - High Energy Physics - Phenomenology,2000-01-03,14
3,hep-ph - High Energy Physics - Phenomenology,2000-01-04,10
4,hep-ph - High Energy Physics - Phenomenology,2000-01-05,16


## 📂 Работа с одной категорией

Для начала сосредоточимся на одной подкатегории. Мы извлечём её временной ряд — упорядочим по дате, сделаем дату индексом и уберём лишние колонки. Это поможет нам удобно считать rolling-статистики и готовить признаки.


In [2]:
def get_category_data(df: pd.DataFrame, category: str):
    category_data = df[df['category'] == category].copy()
    category_data = category_data.sort_values('submitted_date')
    category_data = category_data.set_index('submitted_date')
    return category_data.drop('category', axis=1)

category_data = get_category_data(train, 'hep-ph - High Energy Physics - Phenomenology')
category_data

,num_papers
submitted_date,
2000-01-01,5
2000-01-02,6
2000-01-03,14
2000-01-04,10
2000-01-05,16
...,...
2025-02-05,32
2025-02-06,29
2025-02-07,25


## 🔮 Расширение данных на будущее

Чтобы наша модель могла делать прогнозы, мы расширим временной ряд на одну неделю вперёд.  

Мы добавим 7 будущих дат без значений — они нужны, чтобы модель могла "заглянуть в будущее" и сгенерировать признаки для предсказания тестовых данных.

Для простоты мы будем прогнозировать только одну следующую неделю - а дальше мы будем просто повторять наши прогнозы

In [3]:
def extend_dataset(category_data, last_train_date, future_weeks_num: int = 0):
    if future_weeks_num > 0:
        future_dates = pd.date_range(
                start=last_train_date + pd.Timedelta(days=1),
                periods=future_weeks_num * 7,
                freq='D'
            )
            
        future = pd.DataFrame(index= future_dates, data={'num_papers': None})
        return pd.concat((category_data, future))

last_train_date=train.submitted_date.max()
category_data = extend_dataset(category_data, last_train_date=last_train_date, future_weeks_num=3)

## 📊 Расчёт rolling-признаков (скользящих аггрегаций)

Теперь посчитаем базовые признаки по скользящему окну (rolling window):

- `rolling_sum_during_week` — сколько статей было опубликовано за последние 7 дней.
- `rolling_max_during_week` — максимум статей этой категории в день за последние 7 дней.
- `rolling_min_during_month` и `rolling_max_during_month` — минимум и максимум за последние 28 дней (месяц).

Эти признаки помогут модели понимать недавнюю активность в категории.


In [4]:
def get_rolling_features(cat_data: pd.DataFrame):
    import numpy as np

    daily_features = pd.DataFrame(index=cat_data.index)
    # Weekly rolling features
    weekly_roll = cat_data['num_papers'].rolling('7D', min_periods=1)
    daily_features['rolling_sum_during_week']  = weekly_roll.sum()
    daily_features['rolling_max_during_week']  = weekly_roll.max()
    daily_features['rolling_min_during_week']  = weekly_roll.min()
    daily_features['rolling_mean_during_week'] = weekly_roll.mean()
    daily_features['rolling_std_during_week']  = weekly_roll.std()

    # Monthly rolling features
    month_roll = cat_data['num_papers'].rolling('28D', min_periods=1)
    daily_features['rolling_sum_during_month']  = month_roll.sum()
    daily_features['rolling_min_during_month']  = month_roll.min()
    daily_features['rolling_max_during_month']  = month_roll.max()
    daily_features['rolling_mean_during_month'] = month_roll.mean()
    daily_features['rolling_std_during_month']  = month_roll.std()

    # Seasonal calendar features
    idx = cat_data.index
    daily_features['week_of_year']   = idx.isocalendar().week
    daily_features['month']          = idx.month
    daily_features['quarter']        = idx.quarter

    # Cyclical encoding
    daily_features['woy_sin']    = np.sin(2 * np.pi * daily_features['week_of_year'] / 52)
    daily_features['woy_cos']    = np.cos(2 * np.pi * daily_features['week_of_year'] / 52)
    daily_features['month_sin']  = np.sin(2 * np.pi * daily_features['month']        / 12)
    daily_features['month_cos']  = np.cos(2 * np.pi * daily_features['month']        / 12)

    return daily_features

rolling_features = get_rolling_features(cat_data=category_data)
rolling_features

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pan

,rolling_sum_during_week,rolling_max_during_week,rolling_min_during_week,rolling_mean_during_week,rolling_std_during_week,rolling_sum_during_month,rolling_min_during_month,rolling_max_during_month,rolling_mean_during_month,rolling_std_during_month,week_of_year,month,quarter,woy_sin,woy_cos,month_sin,month_cos
2000-01-01,5.0,5.0,5.0,5.000000,NaN,5.0,5.0,5.0,5.000000,NaN,52,1,1,0.0,1.0,0.500000,8.660254e-01
2000-01-02,11.0,6.0,5.0,5.500000,0.707107,11.0,5.0,6.0,5.500000,0.707107,52,1,1,0.0,1.0,0.500000,8.660254e-01
2000-01-03,25.0,14.0,5.0,8.333333,4.932883,25.0,5.0,14.0,8.333333,4.932883,1,1,1,0.120537,0.992709,0.500000,8.660254e-01
2000-01-04,35.0,14.0,5.0,8.750000,4.112988,35.0,5.0,14.0,8.750000,4.112988,1,1,1,0.120537,0.992709,0.500000,8.660254e-01
2000-01-05,51.0,16.0,5.0,10.200000,4.816638,51.0,5.0,16.0,10.200000,4.816638,1,1,1,0.120537,0.992709,0.500000,8.660254e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-02-26,NaN,NaN,NaN,NaN,NaN,202.0,5.0,32.0,18.363636,10.141723,9,2,1,0.885456,0.464723,0.866025,5.000000e-01
2025-02-27,NaN,NaN,NaN,NaN,NaN,174.0,5.0,32.0,17.400000,10.145607,9,2,1,0.885456,0.464723,0.866025,5.000000e-01
2025-02-28,NaN,NaN,NaN,NaN,NaN,160.0,5.0,32.0,17.777778,10.686180,9,2,1,0.885456,0.464723,0.866025,5.000000e-01
2025-03-01,NaN,NaN,NaN,NaN,NaN,150.0,5.0,32.0,18.750000,10.990255,9,3,1,0.885456,0.464723,1.000000,6.123234e-17


In [5]:
def add_interaction_features(features: pd.DataFrame):
    import numpy as np
    # Basic interactions
    features['interaction_sum_max_week']    = features['rolling_sum_during_week'] * features['rolling_max_during_week']
    features['interaction_sum_max_month']   = features['rolling_sum_during_month'] * features['rolling_max_during_month']
    features['interaction_min_max_week']    = features['rolling_max_during_week'] - features['rolling_min_during_week']
    features['interaction_min_max_month']   = features['rolling_max_during_month'] - features['rolling_min_during_month']

    # Safe denominators to avoid division by zero
    min_week_denom = features['rolling_min_during_week'].abs().clip(lower=1e-6)
    std_month_denom = features['rolling_std_during_month'].abs().clip(lower=1e-6)

    # Ratio interactions
    features['interaction_ratio_max_min_week']   = features['rolling_max_during_week'] / min_week_denom
    features['interaction_ratio_mean_std_month'] = features['rolling_mean_during_month'] / std_month_denom

    # Difference between weekly and monthly sums
    features['interaction_diff_sum_week_month']  = features['rolling_sum_during_week'] - features['rolling_sum_during_month']

    # Seasonal interactions using week-of-year and month encodings
    features['interaction_seasonal_woy_mean_week']  = features['woy_sin'] * features['rolling_mean_during_week']
    features['interaction_seasonal_month_sum_month'] = features['month_sin'] * features['rolling_sum_during_month']

    return features




## ⏪ Добавление лагов (значений сдвига)

Теперь добавим **лаг-признаки** — значения, которые были ровно неделю назад:

- Например, `rolling_sum_during_week_last_week` — это сумма за неделю, но из **предыдущей** недели.
- Такие признаки важны, чтобы модель могла "вспоминать", что происходило ранее, и делать выводы на основе динамики.

Мы используем `.shift(freq='7D')`, чтобы сместить значения на 7 дней назад.

In [6]:
def add_lag_features(features: pd.DataFrame):
    import numpy as np
    new_features = features.copy()
    for col in features.columns:
        # Add lag features for multiple week offsets
        for lag in [1, 2, 4, 8, 14]:
            lag_col = f'{col}_lag_{lag}_week'
            new_features[lag_col] = features[col].shift(freq=pd.Timedelta(days=7 * lag))

        # Drop rows with NaNs introduced by lags
        new_features.dropna(inplace=True)

        # Interaction of current and lagged: difference and safe ratio
        for lag in [1, 2, 4, 8, 14]:
            lag_col = f'{col}_lag_{lag}_week'
            if lag_col in new_features.columns:
                # difference
                new_features[f'{col}_lag_{lag}_week_interaction'] = features[col] - new_features[lag_col]
                # safe ratio denominator
                denom = new_features[lag_col].abs().clip(lower=1e-6)
                new_features[f'{col}_lag_{lag}_week_ratio'] = features[col] / denom
    
    return new_features

lag_features = add_lag_features(rolling_features)
lag_features

,rolling_sum_during_week,rolling_max_during_week,rolling_min_during_week,rolling_mean_during_week,rolling_std_during_week,rolling_sum_during_month,rolling_min_during_month,rolling_max_during_month,rolling_mean_during_month,rolling_std_during_month,...,month_cos_lag_1_week_interaction,month_cos_lag_1_week_ratio,month_cos_lag_2_week_interaction,month_cos_lag_2_week_ratio,month_cos_lag_4_week_interaction,month_cos_lag_4_week_ratio,month_cos_lag_8_week_interaction,month_cos_lag_8_week_ratio,month_cos_lag_14_week_interaction,month_cos_lag_14_week_ratio
2000-04-09,101.0,22.0,5.0,14.428571,6.900656,381.0,0.0,27.0,13.607143,7.410400,...,0.0,-1.0,-0.500000,-500000.00000,-0.500000,-500000.00000,-1.0,-1.0,-1.366025,-0.57735
2000-04-10,100.0,22.0,5.0,14.285714,6.872998,381.0,0.0,27.0,13.607143,7.410400,...,0.0,-1.0,-0.500000,-500000.00000,-0.500000,-500000.00000,-1.0,-1.0,-1.366025,-0.57735
2000-04-11,90.0,22.0,5.0,12.857143,5.984106,376.0,0.0,27.0,13.428571,7.385815,...,0.0,-1.0,-0.500000,-500000.00000,-0.500000,-500000.00000,-1.0,-1.0,-1.366025,-0.57735
2000-04-12,93.0,22.0,5.0,13.285714,5.936168,371.0,0.0,27.0,13.250000,7.306136,...,0.0,-1.0,-0.500000,-500000.00000,-0.500000,-500000.00000,-1.0,-1.0,-1.366025,-0.57735
2000-04-13,87.0,18.0,5.0,12.428571,4.790864,365.0,0.0,27.0,13.035714,7.125760,...,0.0,-1.0,-0.500000,-500000.00000,-0.500000,-500000.00000,-1.0,-1.0,-1.366025,-0.57735
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-02-10,120.0,32.0,5.0,20.000000,11.661904,560.0,5.0,32.0,20.740741,8.733633,...,0.0,1.0,-0.366025,0.57735,-0.366025,0.57735,-0.5,0.5,-0.366025,0.57735
2025-02-11,97.0,32.0,5.0,19.400000,12.934450,533.0,5.0,32.0,20.500000,8.814760,...,0.0,1.0,-0.366025,0.57735,-0.366025,0.57735,-0.5,0.5,-0.366025,0.57735
2025-02-12,65.0,29.0,5.0,16.250000,12.526638,509.0,5.0,32.0,20.360000,8.966976,...,0.0,1.0,-0.366025,0.57735,-0.366025,0.57735,-0.5,0.5,-0.366025,0.57735
2025-02-13,36.0,25.0,5.0,12.000000,11.269428,485.0,5.0,32.0,20.208333,9.127022,...,0.0,1.0,-0.366025,0.57735,-0.366025,0.57735,-0.5,0.5,-0.366025,0.57735


## 📆 Преобразование в недельный уровень

Поскольку нам нужно делать прогноз **по неделям**, агрегируем ежедневные признаки в недельные.

- Каждому дню мы сопоставим конец недели (воскресенье).
- Затем для каждой недели возьмём **последнее доступное значение** признаков.
- Это даст нам ровно одну строку признаков на каждую неделю.

Теперь мы готовы строить таргет и обучать модель.


In [7]:
def build_weekly_features(features):
    daily_features = features.reset_index(names="day")
    daily_features['week'] = daily_features['day'] + pd.to_timedelta(6 - daily_features['day'].dt.weekday, unit='D')
    weekly_features = daily_features.groupby('week').last().reset_index()
    return weekly_features.drop('day', axis=1)

weekly_features = build_weekly_features(lag_features)
weekly_features

,week,rolling_sum_during_week,rolling_max_during_week,rolling_min_during_week,rolling_mean_during_week,rolling_std_during_week,rolling_sum_during_month,rolling_min_during_month,rolling_max_during_month,rolling_mean_during_month,...,month_cos_lag_1_week_interaction,month_cos_lag_1_week_ratio,month_cos_lag_2_week_interaction,month_cos_lag_2_week_ratio,month_cos_lag_4_week_interaction,month_cos_lag_4_week_ratio,month_cos_lag_8_week_interaction,month_cos_lag_8_week_ratio,month_cos_lag_14_week_interaction,month_cos_lag_14_week_ratio
0,2000-04-09,101.0,22.0,5.0,14.428571,6.900656,381.0,0.0,27.0,13.607143,...,0.000000,-1.000000,-0.500000,-500000.000000,-0.500000,-500000.000000,-1.000000e+00,-1.000000,-1.366025,-0.577350
1,2000-04-16,83.0,16.0,6.0,11.857143,3.670993,373.0,1.0,27.0,13.321429,...,0.000000,-1.000000,0.000000,-1.000000,-0.500000,-500000.000000,-1.000000e+00,-1.000000,-1.366025,-0.577350
2,2000-04-23,73.0,21.0,0.0,10.428571,7.590721,359.0,0.0,27.0,12.821429,...,0.000000,-1.000000,0.000000,-1.000000,-0.500000,-500000.000000,-1.000000e+00,-1.000000,-1.366025,-0.577350
3,2000-04-30,93.0,26.0,3.0,13.285714,8.635475,350.0,0.0,26.0,12.500000,...,0.000000,-1.000000,0.000000,-1.000000,0.000000,-1.000000,-5.000000e-01,-500000.000000,-1.366025,-0.577350
4,2000-05-07,66.0,15.0,0.0,9.428571,5.740416,315.0,0.0,26.0,11.250000,...,-0.366025,-1.732051,-0.366025,-1.732051,-0.366025,-1.732051,-8.660254e-01,-866025.403784,-1.732051,-1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1293,2025-01-19,145.0,29.0,7.0,20.714286,8.596788,541.0,4.0,47.0,19.321429,...,0.000000,1.000000,0.000000,1.000000,-0.133975,0.866025,3.330669e-16,1.000000,0.366025,1.732051
1294,2025-01-26,157.0,31.0,12.0,22.428571,7.091242,550.0,4.0,47.0,19.642857,...,0.000000,1.000000,0.000000,1.000000,-0.133975,0.866025,-1.339746e-01,0.866025,0.366025,1.732051
1295,2025-02-02,139.0,29.0,7.0,19.857143,9.441550,564.0,7.0,31.0,20.142857,...,-0.366025,0.577350,-0.366025,0.577350,-0.366025,0.577350,-5.000000e-01,0.500000,0.000000,1.000000
1296,2025-02-09,143.0,32.0,5.0,20.428571,10.706029,584.0,5.0,32.0,20.857143,...,0.000000,1.000000,-0.366025,0.577350,-0.366025,0.577350,-5.000000e-01,0.500000,-0.366025,0.577350


## 🎯 Построение целевой переменной (target)

Теперь создадим **таргет** — количество публикаций в каждой неделе, сдвинутое на `week_horizon`.

- Если `week_horizon=1`, мы будем предсказывать число публикаций **на следующей неделе**.
- Мы агрегируем данные по неделям (`resample('W')`) и сдвигаем их вверх (`shift(-1)`), чтобы каждая неделя "смотрела в будущее".

Это наша целевая переменная для обучения модели.


In [8]:
def build_targets(category_data, week_horizon: int):
    targets = category_data.resample('W').sum().shift(-week_horizon).num_papers.rename('target')
    targets.index.name = 'week'
    return targets

targets = build_targets(category_data=category_data, week_horizon=1)
targets

week
2000-01-02      67
2000-01-09     105
2000-01-16     102
2000-01-23     108
2000-01-30     101
              ... 
2025-02-02     143
2025-02-09       0
2025-02-16       0
2025-02-23       0
2025-03-02    None
Freq: W-SUN, Name: target, Length: 1314, dtype: object

# Итак по одной категории у нас уже есть готовый датасет для тренировки!

In [9]:
current_data = weekly_features.merge(targets, on='week')
current_data

,week,rolling_sum_during_week,rolling_max_during_week,rolling_min_during_week,rolling_mean_during_week,rolling_std_during_week,rolling_sum_during_month,rolling_min_during_month,rolling_max_during_month,rolling_mean_during_month,...,month_cos_lag_1_week_ratio,month_cos_lag_2_week_interaction,month_cos_lag_2_week_ratio,month_cos_lag_4_week_interaction,month_cos_lag_4_week_ratio,month_cos_lag_8_week_interaction,month_cos_lag_8_week_ratio,month_cos_lag_14_week_interaction,month_cos_lag_14_week_ratio,target
0,2000-04-09,101.0,22.0,5.0,14.428571,6.900656,381.0,0.0,27.0,13.607143,...,-1.000000,-0.500000,-500000.000000,-0.500000,-500000.000000,-1.000000e+00,-1.000000,-1.366025,-0.577350,83
1,2000-04-16,83.0,16.0,6.0,11.857143,3.670993,373.0,1.0,27.0,13.321429,...,-1.000000,0.000000,-1.000000,-0.500000,-500000.000000,-1.000000e+00,-1.000000,-1.366025,-0.577350,73
2,2000-04-23,73.0,21.0,0.0,10.428571,7.590721,359.0,0.0,27.0,12.821429,...,-1.000000,0.000000,-1.000000,-0.500000,-500000.000000,-1.000000e+00,-1.000000,-1.366025,-0.577350,93
3,2000-04-30,93.0,26.0,3.0,13.285714,8.635475,350.0,0.0,26.0,12.500000,...,-1.000000,0.000000,-1.000000,0.000000,-1.000000,-5.000000e-01,-500000.000000,-1.366025,-0.577350,66
4,2000-05-07,66.0,15.0,0.0,9.428571,5.740416,315.0,0.0,26.0,11.250000,...,-1.732051,-0.366025,-1.732051,-0.366025,-1.732051,-8.660254e-01,-866025.403784,-1.732051,-1.000000,94
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1293,2025-01-19,145.0,29.0,7.0,20.714286,8.596788,541.0,4.0,47.0,19.321429,...,1.000000,0.000000,1.000000,-0.133975,0.866025,3.330669e-16,1.000000,0.366025,1.732051,157
1294,2025-01-26,157.0,31.0,12.0,22.428571,7.091242,550.0,4.0,47.0,19.642857,...,1.000000,0.000000,1.000000,-0.133975,0.866025,-1.339746e-01,0.866025,0.366025,1.732051,139
1295,2025-02-02,139.0,29.0,7.0,19.857143,9.441550,564.0,7.0,31.0,20.142857,...,0.577350,-0.366025,0.577350,-0.366025,0.577350,-5.000000e-01,0.500000,0.000000,1.000000,143
1296,2025-02-09,143.0,32.0,5.0,20.428571,10.706029,584.0,5.0,32.0,20.857143,...,1.000000,-0.366025,0.577350,-0.366025,0.577350,-5.000000e-01,0.500000,-0.366025,0.577350,0


## 🏗️ Сбор финального датасета для обучения

Теперь объединим всё вместе для всех категорий:

1. Для каждой категории:
   - Извлекаем временной ряд.
   - Добавляем будущую неделю для генерации признаков.
   - Считаем rolling-признаки.
   - Добавляем лаги (предыдущие недели).
   - Агрегируем всё на недельном уровне.
   - Создаём `target` — количество публикаций на следующей неделе.
2. Объединяем данные всех категорий в один финальный `dataset`.
3. Преобразуем колонку `category` в категориальный тип (для LightGBM).

Теперь у нас есть полноценный обучающий набор, готовый для машинного обучения! 💪


In [10]:
from tqdm.auto import tqdm

last_train_date=train.submitted_date.max()
progress_bar = tqdm(train.category.unique())

dataset = []
for category in progress_bar:
    category_data = get_category_data(train, category)
    extended_category_data = extend_dataset(category_data, last_train_date=last_train_date, future_weeks_num=1)
    rolling_features = get_rolling_features(cat_data=extended_category_data)
    rolling_features = add_interaction_features(rolling_features)
    lag_features = add_lag_features(rolling_features)
    weekly_features = build_weekly_features(lag_features)
    targets = build_targets(category_data=category_data, week_horizon=1)
    data = weekly_features.merge(targets, on='week')
    data['category'] = category
    dataset.append(data)

dataset = pd.concat(dataset)
dataset['category'] = dataset['category'].astype('category')
dataset

  0%|          | 0/140 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in greater_equal
  return op(a, b)
/usr/local/lib/python3.11/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in greater_equal
  return op(a, b)
/usr/local/lib/python3.11/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in greater_equal
  return op(a, b)
/usr/local/lib/python3.11/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in greater_equal
  return op(a, b)
/usr/local/lib/python3.11/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in greater_equal
  return op(a, b)
/usr/local/lib/python3.11/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in greater_equal
  return op(a, b)
/usr/local/lib/python3.11/dist-packages/

,week,rolling_sum_during_week,rolling_max_during_week,rolling_min_during_week,rolling_mean_during_week,rolling_std_during_week,rolling_sum_during_month,rolling_min_during_month,rolling_max_during_month,rolling_mean_during_month,...,interaction_seasonal_month_sum_month_lag_2_week_interaction,interaction_seasonal_month_sum_month_lag_2_week_ratio,interaction_seasonal_month_sum_month_lag_4_week_interaction,interaction_seasonal_month_sum_month_lag_4_week_ratio,interaction_seasonal_month_sum_month_lag_8_week_interaction,interaction_seasonal_month_sum_month_lag_8_week_ratio,interaction_seasonal_month_sum_month_lag_14_week_interaction,interaction_seasonal_month_sum_month_lag_14_week_ratio,target,category
0,2000-04-09,101.0,22.0,5.0,14.428571,6.900656,381.0,0.0,27.0,13.607143,...,-14.044321,9.591735e-01,-16.044321,9.536291e-01,-22.516660,9.361179e-01,324.455679,59.991942,83.0,hep-ph - High Energy Physics - Phenomenology
1,2000-04-16,83.0,16.0,6.0,11.857143,3.670993,373.0,1.0,27.0,13.321429,...,1.732051,1.005391e+00,-18.972524,9.445248e-01,-23.382686,9.325000e-01,284.027476,8.282756,73.0,hep-ph - High Energy Physics - Phenomenology
2,2000-04-23,73.0,21.0,0.0,10.428571,7.590721,359.0,0.0,27.0,12.821429,...,-19.052559,9.422572e-01,-33.096880,9.037881e-01,-15.588457,9.522546e-01,219.403120,3.397848,93.0,hep-ph - High Energy Physics - Phenomenology
3,2000-04-30,93.0,26.0,3.0,13.285714,8.635475,350.0,0.0,26.0,12.500000,...,-19.918584,9.383378e-01,-18.186533,9.433962e-01,-47.891109,8.635581e-01,160.608891,2.127080,66.0,hep-ph - High Energy Physics - Phenomenology
4,2000-05-07,66.0,15.0,0.0,9.428571,5.740416,315.0,0.0,26.0,11.250000,...,-153.403120,5.065887e-01,-172.455679,4.773368e-01,-188.500000,4.552023e-01,-33.500000,0.824607,94.0,hep-ph - High Energy Physics - Phenomenology
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1292,2025-01-12,62.0,11.0,7.0,8.857143,1.573592,288.0,5.0,24.0,10.285714,...,144.000000,1.440000e+08,144.000000,1.440000e+08,353.000000,6.889952e-01,505.132593,0.398746,65.0,q-bio
1293,2025-01-19,65.0,15.0,4.0,9.285714,4.309458,255.0,4.0,15.0,9.107143,...,-33.500000,7.919255e-01,127.500000,1.275000e+08,334.000000,6.174334e-01,486.900543,0.354757,79.0,q-bio
1294,2025-01-26,79.0,15.0,8.0,11.285714,2.288689,266.0,4.0,15.0,9.500000,...,-11.000000,9.236111e-01,133.000000,1.330000e+08,133.000000,1.330000e+08,515.783228,0.347455,84.0,q-bio
1295,2025-02-02,84.0,20.0,3.0,12.000000,5.597619,290.0,3.0,20.0,10.357143,...,123.647367,1.969783e+00,90.147367,1.559922e+00,251.147367,2.511474e+08,625.270342,0.671296,90.0,q-bio


## 🧪 Разделение на обучающую, валидационную и тестовую выборки

Перед обучением модели нужно правильно разделить данные:

- **train_dataset** — всё до последних 4 недель (обучение).
- **valid_dataset** — последние 4 недели перед последней известной датой (валидация).
- **test_dataset** — данные без таргета (будущее, на которое мы будем делать предсказание).

Это разделение имитирует реальную ситуацию: мы учимся на прошлом, проверяем на последнем известном фрагменте, и делаем прогноз на будущее.


In [11]:
labeled_data = dataset[dataset.target.notnull()].reset_index(drop=True).dropna()

n_valid_weeks = 4
valid_start_date = last_train_date - pd.Timedelta(days=7 * n_valid_weeks)

print(f"Предпоследняя неделя: {valid_start_date}")
valid_dataset = labeled_data[labeled_data.week >= valid_start_date]
train_dataset = labeled_data[labeled_data.week < valid_start_date]
test_dataset = dataset[dataset.target.isnull()].reset_index(drop=True)
train_dataset_copy = train_dataset.copy()

print(train_dataset.shape, valid_dataset.shape, test_dataset.shape)

Предпоследняя неделя: 2025-01-12 00:00:00
(173524, 419) (560, 419) (140, 419)


## 📦 Подготовка данных для LightGBM

Теперь подготовим данные в нужном формате для обучения модели:

- `train_set` и `valid_set` создаются в формате `lightgbm.Dataset` — это специальный формат для ускорения обучения.
- `test_set` остаётся обычным `DataFrame`, потому что мы просто хотим получить предсказания.

Колонку `target` мы используем только как целевую переменную (`label`).


In [12]:
import lightgbm

train_set = lightgbm.Dataset(train_dataset.drop(['week', 'target'], axis=1), label=train_dataset['target'])
valid_set = lightgbm.Dataset(valid_dataset.drop(['week', 'target'], axis=1), label=valid_dataset['target'])
test_set = test_dataset.drop(['week', 'target'], axis=1)

## 🌲 Обучение модели LightGBM с кастомной метрикой

Теперь мы обучим модель LightGBM для задачи регрессии:

- Используем метрику `Safe MAPE` — она помогает избежать слишком большого штрафа на малых значениях.
- Указываем параметры модели:
  - `objective: regression` — мы предсказываем количество статей (то есть числа).
  - `learning_rate: 0.05` — насколько быстро обучается модель.
  - `depth: 5` — максимальная глубина дерева.
  - `metric: None` — мы используем свою метрику, не встроенную.

Мы также включаем:
- **Раннюю остановку** (`early_stopping`), чтобы не переобучиться.
- **Логгинг** каждые 50 итераций, чтобы отслеживать процесс обучения.

Параметры тренировки были выбраны случайно, просто чтобы натренировать хоть что-то

In [13]:
def safe_mape_lgb(y_pred, dataset):
    y_true = dataset.get_label()
    denominator = pd.Series(y_true).abs().clip(lower=10.0)
    error = abs(y_pred - y_true) / denominator
    return 'safe_mape', error.mean(), False  # False = the lower the better

# 2. Параметры модели
params = {
    'objective': 'regression',
    'learning_rate': 0.05,
    'depth': 5,
    'verbosity': -1,
    'metric': 'None'
}
# Добавляем cross-validation с TimeSeriesSplit
from sklearn.model_selection import TimeSeriesSplit
import numpy as np

X = train_dataset_copy.drop(['week', 'target'], axis=1)
y = train_dataset_copy['target']

tscv = TimeSeriesSplit(n_splits=5)
cv_scores = []

for fold, (train_idx, val_idx) in enumerate(tscv.split(X), 1):
    X_tr, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_tr, y_val = y.iloc[train_idx], y.iloc[val_idx]

    lgb_tr = lightgbm.Dataset(X_tr, label=y_tr)
    lgb_val = lightgbm.Dataset(X_val, label=y_val)

    cv_model = lightgbm.train(
        params,
        lgb_tr,
        num_boost_round=200,
        valid_sets=[lgb_val],
        valid_names=['val'],
        feval=safe_mape_lgb,
        callbacks=[lightgbm.early_stopping(stopping_rounds=50)]
    )

    preds = cv_model.predict(X_val)
    score = safe_mape_lgb(preds, lgb_val)[1]
    cv_scores.append(score)
    print(f'Fold {fold} Safe-MAPE: {score:.4f}')

print(f'Mean CV Safe-MAPE: {np.mean(cv_scores):.4f}')

# 3. Обучение с кастомной метрикой
model = lightgbm.train(
    params,
    train_set,
    num_boost_round=200,
    valid_sets=[valid_set],
    valid_names=['valid'],
    feval=safe_mape_lgb,
    callbacks=[
        lightgbm.early_stopping(stopping_rounds=50),
        lightgbm.log_evaluation(period=50)
    ]
)


Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[200]	val's safe_mape: 0.205166
Fold 1 Safe-MAPE: 0.2052
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[199]	val's safe_mape: 0.203672
Fold 2 Safe-MAPE: 0.2037
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[198]	val's safe_mape: 0.190592
Fold 3 Safe-MAPE: 0.1906
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[200]	val's safe_mape: 0.176814
Fold 4 Safe-MAPE: 0.1768
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[200]	val's safe_mape: 0.1471
Fold 5 Safe-MAPE: 0.1471
Mean CV Safe-MAPE: 0.1847
Training until validation scores don't improve for 50 rounds
[50]	valid's safe_mape: 0.209964
[100]	valid's safe_mape: 0.194338
[150]	valid's safe

# Используем нашу натренированную модель на тестовых данных

In [14]:
test_dataset['predicted'] = model.predict(test_set)
test_dataset[['category', 'predicted']]

,category,predicted
0,hep-ph - High Energy Physics - Phenomenology,144.895578
1,math.CO - Combinatorics,109.791112
2,cs.CG - Computational Geometry,10.097793
3,physics.gen-ph - General Physics,5.433011
4,math.CA - Classical Analysis and ODEs,30.325404
...,...,...
135,eess.IV - Image and Video Processing,90.390950
136,eess.SP - Signal Processing,109.862966
137,q-fin.MF - Mathematical Finance,5.759969
138,cond-mat,409.051452


# Делаем финальный сабмишен

проверяем что наш submission упорядочен также как sample_submission.csv

In [15]:

sample_submission = pd.read_csv(os.path.join(work_dir, "sample_submission.csv"))
sample_submission['category'] = sample_submission['id'].apply(lambda x: x.split('__')[0])
sample_submission = sample_submission.merge(test_dataset[['category', 'predicted']], on='category')
sample_submission[['id', 'predicted']].to_csv('submission.csv', index=False)